In [2]:
import requests
import time
import json

from inspirehep import *

In [3]:
f=open('data/db_LA.json','r')
db=json.load(f)
f.close()
idsLA=[d.get('LA_institution_id') for d in db]
idsAU=[d.get('author_id') for d in db]
db_index=get_index(db)

In [4]:
db_inst_index=get_index(db,index='LA_institution_id')

Pagination

In [ ]:
#page_max=float('inf')
page=0
emergency_exit=300
mp=0
LONG_URL=False
while page>=0:
    if page_max<13 or LONG_URL: #Only few pages left or too long URL. Go through all of them
        #collaborations=collaborations[:102]
        collaborations_old=collaborations #last 10 pages
    if mp>=emergency_exit:
        print('Emergency Exit!')
        break
    mp=mp+1
    if len(collaborations)>0 and  collaborations_old!=collaborations:
        cn_search='https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=ac>100+and+de>2018'
        cn=f"{cn_search}+and+not+cn+{'+and+not+cn+'.join(collaborations)}".replace(' ','+')
        collaborations_old=collaborations
        page=0
        if len(cn)>LONG_URL:
            LONG_URL=True
    else:
        #increase pagination
        if len(collaborations)==0:
            extra=''
        else:
            extra='+and+not+cn+'
        page=page+1
        cn_search=f'https://inspirehep.net/api/literature?sort=mostrecent&size=25&page={page}&q=ac>100+and+de>2018'
        cn=f"{cn_search}{extra}{'+and+not+cn+'.join(collaborations[:102])}".replace(' ','+')

    print(cn)
    time.sleep(sleep)
    r=requests.get(cn)
    d=r.json()
    page_max=int(d.get('hits').get('total')/25)+1
    if page>page_max:
        break

    l=d.get('hits').get('hits')

    #Not affilation in `'authors'` list for collaborations!

    for ld in l:
        lit=ld.get('metadata')
        if not lit.get('collaborations'):
            continue # next paper
        try:
            collaboration=lit.get('collaborations')[0].get('value')
        except:
            collaboration=''

        if collaboration in collaborations:
            continue #next paper

        print(f'collaboration → {collaboration}')
        la=lit.get('authors')
        ii=0
        for a in la:
            FoS=[collaboration]
            try:
                PAC=lit['primary_arxiv_category']
            except:
                PAC=[]
            
            print(ii,end='\r')
            ii=ii+1
            CONTINUE,author_id,url_author,FoS,PAC,db=get_work(db,a,idsLA,idsAU,db_index,FoS,PAC)
            if CONTINUE:
                continue #next author
            positions,email_addresses,name,aids=get_author(url_author) #USES API!
            try:
                inst_id=[d.get('record').get('$ref').split('/')[-1] 
                     for d in positions if d.get('record') and d.get('current')][0]
            except:
                inst_id=None
            if inst_id not in idsLA:
                continue #next author
            di=db[db_inst_index[inst_id]]

            new={'author_id':author_id,
                     'name':name,
                     'LA_institution_id':inst_id,
                     'external_system_identifiers':di.get('external_system_identifiers'),
                     'LA_country':di.get('LA_country'),
                     'LA_institution':di.get('LA_institution'),
                     'email_addresses':email_addresses,
                     'positions':positions,
                     'ids':aids,
                     'primary_arxiv_category':PAC,
                     'Fields of Study': FoS # in ['Particle physics','Cosmology','Astrophysics','Mathematical physics']+collaborations
                    }
            db.append(new)
            #raise Exception('STOP')            
            #break
        collaborations.append(collaboration) #Only after search for all the authors
        f=open('db_LA.json','w')
        json.dump(db,f)
        f.close()    
        #==============
        #break            